In [1]:
import os
import os.path as op
import json
import cv2
import base64
import random
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from PIL import Image, ImageDraw

from maskrcnn_benchmark.structures.tsv_file import TSVFile

In [2]:
orig_root = '/media/stopmosk/data/huawei/datasets_orig/BIG'
# exp_root = '../../../../datasets_proc/big_nn_nms1'

In [3]:
with open(op.join(orig_root, 'Final_EN.json')) as f:
    dataset_captions = json.load(f)

dataset_captions = dataset_captions['annotations']
len(dataset_captions)

2620065

In [4]:
dataset_captions[0]

{'file_name': 'a1650e00b6261e99a6bbe6fe13919302.jpg',
 'caption': 'author : a life in photography -- in pictures',
 'category_id': -1}

In [5]:
sorted_caps = sorted(dataset_captions, key=lambda x: x['caption'])

only_captions = [s['caption'] for s in dataset_captions]
only_captions.sort()
only_captions[120000]

In [ ]:
# Sample images 

plt.figure(figsize=(8, 8))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(np.clip(train_data[i], 0, 1))
    plt.xlabel('plane' if train_labels[i] else 'bird')
plt.show()

In [41]:
START = 1
END = 2
ANY = 3
TOEND = 4  # Find first and replace to end  
# string "table -- stock photo..." with rule (TOEND, ' -- stock', '') gets "table"

rules = [
    (START, ' Chefs ', 'Dicks '),
    (START, '22', None),
    (TOEND, '- stock photo', '')
]

In [ ]:
strings = get_from_file('capss.txt')
rules = []


def parse_raw(raw_str):
    sub_s, rep_s, tmp_s = None, None, None
    rule_type = START

    i = 0
    is_text = True
    accum = ''
    while True:
        c = raw_str[i]
        
        if is_text:
            if c == opener:
                is_text = False
                tmp_s = accum
                accum = ''
            else:
                accum += c
            i += 1
            continue

        # else we're in ctrl_mode
        if c == "'" or c == '"':
            opener = c
            is_text = True
        elif c == '-':
            if raw_str[i: i+3] != '-->':
                print('ERROR':, raw_str)
                exit
            sub_s = tmp_s
            i += 3            
            continue
        elif c == ' ':
            pass            
        elif c == 'e':
            rule_type = END
        elif c == 'a':
            rule_type = ANY            
        elif c == 't':
            rule_type = TOEND        
        else:
            print(raw_str, c)
            
        i += 1
        if i == len(raw_str):
            # some finish process
            if sub_s is None:
                sub_s = tmp_s
            else:
                rep_s = tmp_s            
            break
            
    return rule_type, sub_s, rep_s


for raw_rule in strings:
    rule = parse_raw(raw_rule)
    rules.append(rule)


In [42]:
def apply_rules(caption, rules):
    # One by one apply rules, repeat until [del] or [same str (no changes)]
    curr_caption = caption
    while True:
        temp_caption = curr_caption

        for (where, sub_str, replace_str) in rules:
            if not ((where == START and curr_caption.startswith(sub_str))
                    or (where == END and curr_caption.endswith(sub_str)) 
                    or (where == ANY and sub_str in curr_caption)
                    or (where == TOEND and sub_str in curr_caption)):
                continue
                            
            # if we have to delete this caption
            if replace_str is None:
                    return None

            # else we should replace substring
            if where == START:
                curr_caption = curr_caption.replace(sub_str, replace_str, 1)
            elif where == END:
                c_len, s_len = len(curr_caption), len(sub_str)
                curr_caption = curr_caption[:c_len - s_len] + replace_str
            elif where == TOEND:
                curr_caption = curr_caption[0: curr_caption.find(sub_str)] + replace_str
            elif where == ANY:
                curr_caption = curr_caption.replace(sub_str, replace_str)
            else:
                raise RuntimeError('Unknown rule')
            
        if curr_caption == temp_caption:
            return curr_caption

filtered_captions = []

for el in tqdm(sorted_caps):
    cap = el['caption']
    new_cap = apply_rules(cap, rules)
    if new_cap is None:
        continue
    new_el = el.copy()
    new_el['caption'] = new_cap
    filtered_captions.append(new_el)

diff = len(sorted_caps) - len(filtered_captions)
print(diff)

100%|██████████| 2620065/2620065 [00:03<00:00, 674386.70it/s]

4495


In [65]:
with open('../../../../caps.txt', 'w') as fp:
    for cap in only_captions:
        fp.write(cap + '\n')